In [ ]:
import xml.etree.ElementTree as ET
import re
import math
import itertools
from google.colab import files

In [ ]:
def get_first_last_coords(d_attr):
    """パスd属性から最初の(M命令)座標(始点)と最後の座標(終点)を抽出"""
    m = re.search(r'[Mm]\s*([\-\d\.]+),([\-\d\.]+)', d_attr, flags=re.IGNORECASE)
    if m:
        x0, y0 = float(m.group(1)), float(m.group(2))
    else:
        x0, y0 = 0.0, 0.0

    coords = re.findall(r'([\-\d\.]+),([\-\d\.]+)', d_attr)
    if coords:
        xN, yN = float(coords[-1][0]), float(coords[-1][1])
    else:
        xN, yN = x0, y0
    return (x0, y0), (xN, yN)

def distance(p1, p2):
    """ユークリッド距離"""
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def reorder_paths_greedy(path_info_list):
    """
    パスの (start, end, 要素) のリストを与えられたとき、
    「最近傍法」で順番を決めて返す。
    - まず最初のパスを確定し、以降は「現在のパスの終点」に
      もっとも近い未訪問パスの始点を持つパスを選ぶ。
    """
    if not path_info_list:
        return []

    # 未訪問リストに全部入れておく
    unvisited = path_info_list[:]
    visited = []

    # ここでは「最初のパス」を単純にリストの先頭にする
    current = unvisited.pop(0)
    visited.append(current)

    while unvisited:
        # 現在のパスの end
        _, current_end, _ = current

        # 距離が最小のパスを見つける
        best_idx = None
        best_dist = float('inf')
        for i, (start_i, end_i, elem_i) in enumerate(unvisited):
            dist = distance(current_end, start_i)
            if dist < best_dist:
                best_dist = dist
                best_idx = i

        # 発見した最も近いパスを訪問
        current = unvisited.pop(best_idx)
        visited.append(current)

    return visited

In [ ]:
###########################
# 1) ColabへSVGファイルをアップロード
###########################
uploaded = files.upload()  # これで input.svg をアップロードする

###########################
# 2) SVGのパース
###########################
tree = ET.parse('input.svg')
root = tree.getroot()

svg_ns = root.tag.split('}')[0].strip('{')

###########################
# 3) グループごとに <path> を「最近傍法」で並び替え
###########################
for g in root.findall(f'{{{svg_ns}}}g'):
    paths = g.findall(f'{{{svg_ns}}}path')
    if len(paths) <= 1:
        continue  # 並び替え不要

    # パス情報 (start, end, element) を格納
    path_info_list = []
    for p in paths:
        d_attr = p.get('d', '')
        s, e = get_first_last_coords(d_attr)
        path_info_list.append((s, e, p))

    # greedy(最近傍)で並び順を決定
    new_order = reorder_paths_greedy(path_info_list)

    # 一旦 g 下の path を削除
    for p in paths:
        g.remove(p)
    # 新しい順で追加
    for (s, e, elem) in new_order:
        g.append(elem)

###########################
# 4) 結果を "optimized.svg" として出力 & ダウンロード
###########################
tree.write('optimized.svg', encoding='utf-8', xml_declaration=True)
files.download('optimized.svg')